In [ ]:
from crewai import Agent, Task, Crew,Process, LLM
from crewai_tools import SerperDevTool
import os
import ollama
from langchain_openai import ChatOpenAI

In [2]:
os.environ["OPENAI_API_KEY"] = "NA"

llm = ChatOpenAI(
    model = "ollama/deepseek-r1",
    base_url = "http://localhost:11434/v1")

In [3]:
transcripts = """
Sarah : Good morning everyone. Thanks for joining our blocker review meeting. As you know, we're aiming to push v2.5 of the app to production next Tuesday. We need to identify and resolve any remaining blockers before then. James, can you start with an overview of where we stand from an engineering perspective?
James : Sure. We've made good progress since our last meeting. The core functionality is stable, but we still have three critical bugs that need resolution. The most pressing is the data synchronization issue when users go offline. Right now, if a user makes changes while offline, about 20% of the time those changes aren't properly synced when they reconnect.
Maya : I can confirm that. We've reproduced it consistently in our testing environment. The problem seems more pronounced on Android than iOS.
Daniel : We've narrowed down the issue. It's related to how the local database handles conflict resolution. I've implemented a potential fix that's currently in the staging environment. Maya, could your team prioritize testing that today?
Maya: Absolutely. We'll make that our top priority.
James: Second issue is the memory leak we identified in the video processing feature. It's not immediately visible to users, but after extended use, it causes the app to slow down significantly and eventually crash.
Daniel: I'm working on that one. It's challenging because it only happens after about 30 minutes of continuous use. I've identified where the resources aren't being properly released, but the fix is complex because it touches several components. I think I can have it resolved by tomorrow afternoon.
Sarah: That's cutting it close to our deadline. Is there a risk we won't make it?
Daniel: There's always some risk, but I'm confident we'll resolve it. If absolutely necessary, we could disable the video processing feature for this release and add it back in v2.6.
Sarah: Let's keep that as Plan B. What's the third issue, James?
James: The payment processing timeout. Under high load, payment confirmations are taking too long, causing users to think their payment failed when it actually succeeded.
Raj (DevOps): That's on me. Our cloud service provider had a partial outage last week that revealed this weakness in our architecture. I've implemented a more robust queueing system with better retry logic, but I'm concerned about our current infrastructure capacity.
Sarah: Can you elaborate?
Raj: Based on our projections, we're expecting a 40% increase in users after this release due to the new features. Our current server configuration won't handle that load effectively. We need to upgrade our infrastructure before the release.
Sarah: That wasn't in our original plan. What's involved?
Raj: We need to scale up our database instances and add more application servers. I've prepared a document with the details, but the bottom line is it will take about two days to implement and test properly.
Sarah: That's significant. Alex, any blockers from the UI/UX side?
Alex (UI/UX): The accessibility audit revealed three critical issues that need to be addressed. The color contrast in the new dashboard doesn't meet WCAG standards, screen readers can't properly navigate the payment forms, and the keyboard navigation is broken in several key workflows.
James: How long will those take to fix?
Alex: I've already prepared the design changes. If Daniel or another developer can pair with me, we could knock them out in a day.
James: I'll assign Li to work with you on that tomorrow.
Sarah: Let's summarize the blockers then:

Data sync issue in offline mode
Memory leak in video processing
Payment processing timeout
Infrastructure scaling for anticipated load
Accessibility compliance issues

Maya: There's one more thing. The performance testing on older devices shows unacceptable load times for the main dashboard. It's taking over 8 seconds on devices that are 3+ years old, which represents about 30% of our user base.
James: Is that because of the new analytics visualizations?
Maya: Yes, the charts library we're using is very resource-intensive.
Alex: We could implement progressive loading or simplify the visualizations for older devices.
James: Progressive loading would be safer at this point. How much work would that be, Daniel?
Daniel: Probably a day of work, and another day for testing.
Sarah: Alright, so we have six critical blockers. Based on the estimates, it sounds like we need at least three more days than we planned. Let's discuss whether we should push back the release date or consider a phased rollout.
Raj: I'd recommend a phased rollout, starting with 10% of users and gradually increasing. That would give us more time to address any issues that emerge and scale the infrastructure appropriately.
Maya: From a quality perspective, I'd be more comfortable with a short delay rather than risking a bad user experience.
James: Agreed. I think we should push the release to Friday of next week instead of Tuesday. That gives us the cushion we need.
Sarah: That makes sense. Let's go with that plan. I'll update stakeholders about the new timeline this afternoon. James, can you send out an updated task breakdown with assignments and deadlines by end of day?
James: Will do.
Sarah: Any other blockers or concerns we should discuss?
Raj: Just a reminder that we need to coordinate with the marketing team about the announcement timing.
Sarah: Good point. I'll handle that as part of my communications today. Anything else?
[Brief silence]
Sarah: Alright, let's get to work then. We'll meet again on Friday for a progress check. Thanks everyone.
"""

In [4]:
meeting_summarizer = Agent(
    role="Meeting Summarizer",
    goal="""Analyze and summarize the key points, decisions, and action items from the provided {meeting_transcripts}.""",
    backstory="""You are an expert at extracting the essence of conversations, identifying key ideas, and presenting concise and actionable summaries. You understand the nuances of meetings, including context, tone, and priorities.""",
    # tools=[SerperDevTool()],  # Add tools if necessary
    llm=llm
)

meeting_summarizing_task = Task(
    description="""Using the provided {meeting_transcripts}, generate a concise and comprehensive summary of the meeting. Focus on key ideas, decisions made, and any follow-up actions required.""",
    expected_output="""A well-structured paragraph summarizing the meeting, highlighting the main discussion points, decisions, and next steps.""",
    agent=meeting_summarizer
)


In [5]:
action_point_generator=Agent(
    role= "Action Point Generator",
    goal="""generate the action points from the {meeting_transcripts}""",
    backstory="""you are great at understanding the context of the meeting and generating the action points""",
    llm=llm
)

In [6]:
action_point_generation_task=Task(
    description="""Generate the actions points from the  {meeting_transcripts} so that it can be discussed in the next meeting.""",
    expected_output="""A pointer based action plans for each user in the meeting to help define the scope of work and the next steeps that will be discussed in the next meeting.""",
    agent=action_point_generator
)

In [8]:

crew=Crew(
    agents=[meeting_summarizer,action_point_generator],
    tasks=[meeting_summarizing_task,action_point_generation_task],
    process=Process.sequential,
    verbose=True
)
result=crew.kickoff(inputs={"meeting_transcripts":transcripts})
print(result)



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: b18a6e53-8339-4970-9184-ea51f1b8274e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 6d36a7da-0a56-4c6f-94dc-fbc275497147
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 6d36a7da-0a56-4c6f-94dc-fbc275497147
       Status: Executing Task...
    └── 🤖 Agent: Meeting Summarizer
            Status: In Progress

# Agent: Meeting Summarizer
## Task: Using the provided 
Sarah : Good morning everyone. Thanks for joining our blocker review meeting. As you know, we're aiming to push v2.5 of the app to production next Tuesday. We need to identify and resolve any remaining blockers before then. James, can you start with an overview of where we stand from an engineering perspective?
James : Sure. We've made good progress since our last meeting. The core functionality is stable, but we still have three critical bugs that need resolution. The most pressing is the data synchronization issue when users go offline. Right now, if a user makes changes while offline, about 20% of the time those changes aren't properly synced when they reconnect.
Maya : I can confirm that. We've reproduced it consistently in our testing environment. The problem seems more pronounced on Android than iOS.
Daniel : We've narrowed down the issue. It's related to how the local database handles conflict resolution. I've implemented 

🚀 Crew: crew
└── 📋 Task: 6d36a7da-0a56-4c6f-94dc-fbc275497147
       Status: Executing Task...
    └── 🤖 Agent: Meeting Summarizer
            Status: ✅ Completed

...

🚀 Crew: crew
├── 📋 Task: 6d36a7da-0a56-4c6f-94dc-fbc275497147
│      Assigned to: Meeting Summarizer
│      Status: ✅ Completed
│   └── 🤖 Agent: Meeting Summarizer
│           Status: ✅ Completed
└── 📋 Task: ac5769c2-b647-493f-92f4-12c7d304c909
       Status: Executing Task...
    └── 🤖 Agent: Action Point Generator
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 6d36a7da-0a56-4c6f-94dc-fbc275497147
│      Assigned to: Meeting Summarizer
│      Status: ✅ Completed
│   └── 🤖 Agent: Meeting Summarizer
│           Status: ✅ Completed
└── 📋 Task: ac5769c2-b647-493f-92f4-12c7d304c909
       Assigned to: Action Point Generator
       Status: ✅ Completed
    └── 🤖 Agent: Action Point Generator
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ac5769c2-b647-493f-92f4-12c7d304c909                                                                     │
│  Agent: Action Point Generator                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: b18a6e53-8339-4970-9184-ea51f1b8274e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

<think>
Alright, I'm trying to summarize this meeting based on the provided notes. Let me go through each part step by step.

First, Raj mentioned an infrastructure issue due to a previous partial outage. He's taking steps to scale up servers but estimates two days for implementation and testing. That seems significant because it's unexpected and could cause delays.

Sarah then brought up performance issues with older devices, specifically the main dashboard loading too slow on devices over three years old. This affects about 30% of users, which is notable. She suggested progressive loading or simplifying charts, but that would take a day plus testing.

Alex raised accessibility concerns: color contrast, screen reader navigation, and keyboard issues. He needs design changes with Daniel's help, potentially taking a day to fix.

James mentioned a phased rollout from 10% users to Friday next week due to the load issues, pushing back the release date by three days from Tuesday to Friday.



In [15]:
print("""<think>
Alright, I need to summarize the meeting based on the provided notes. Let me go through each section step by step.

First, Raj mentioned an infrastructure issue due to a previous partial outage. He's taking steps to scale up servers but estimates two days for implementation and testing. That seems significant because it's unexpected and could cause delays.

Sarah then brought up performance issues with older devices, specifically the main dashboard loading too slow on devices over three years old. This affects about 30% of users, which is notable. She suggested progressive loading or simplifying charts, but that would take a day plus testing.

Alex raised accessibility concerns: color contrast, screen reader navigation, and keyboard issues. He needs design changes with Daniel's help, potentially taking a day to fix.

James mentioned a phased rollout from 10% users to Friday next week due to the load issues, pushing back the release date by three days from Tuesday to Friday.

Sarah also discussed coordinating with the marketing team about the announcement timing, which she'll handle after updating them.

So, putting it all together: The meeting identified multiple critical issues—infrastructure scaling, performance on older devices, accessibility, and a phased rollout. Raj suggested pushing back the release to Friday due to these factors, aligning with user feedback and strategic planning.
</think>

The meeting identified several critical challenges impacting the upcoming release of version 2.0.
Key issues include infrastructure scalability to handle increased user load, performance degradation on older devices (3+ years old), accessibility compliance concerns for key functionalities, 
and the need for a phased rollout strategy from 10% user adoption to full release by Friday next week. Raj proposed delaying the release due to these factors, with an estimated three additional
days needed beyond the initial timeline. Sarah emphasized the importance of prioritizing user experience and quality over speed, coordinating with the marketing team to adjust announcement timing
accordingly. The discussion highlighted the need for a phased approach to ensure smooth scaling while addressing technical and accessibility issues.

""")

<think>
Alright, I need to summarize the meeting based on the provided notes. Let me go through each section step by step.

First, Raj mentioned an infrastructure issue due to a previous partial outage. He's taking steps to scale up servers but estimates two days for implementation and testing. That seems significant because it's unexpected and could cause delays.

Sarah then brought up performance issues with older devices, specifically the main dashboard loading too slow on devices over three years old. This affects about 30% of users, which is notable. She suggested progressive loading or simplifying charts, but that would take a day plus testing.

Alex raised accessibility concerns: color contrast, screen reader navigation, and keyboard issues. He needs design changes with Daniel's help, potentially taking a day to fix.

James mentioned a phased rollout from 10% users to Friday next week due to the load issues, pushing back the release date by three days from Tuesday to Friday.

Sa